# `git2net` - Extracting and analysing co-editing relationships from *git* repositories

In this tutorial you will learn the basic steps required to obtain a co-editing relationship from a git repoitory using `git2net`.

## Prerequisits

This tutorial assumes you have `git2net` installed. In addition, it is recommended to create a folder for this tutorial as additional files will be downloaded to your local directory (if not specified otherwise).

## Repository Mining

To start, you will need to select and clone a git repository that you are interested in analysing. For the purpose of this tutorial, we will analyse the repository behind `git2net`&mdash;aiming to finally find a solution to the well-known chicken and egg problem.

The following lines will clone the `git2net` repository to your current working directory. To change this location, you can edit the path to the local directory stored in `local_directory`. The folder name of the repository is the name of the repository, which we store in `repo_name`.

In [1]:
import git
import os
import shutil

git_repo_url = 'https://github.com/gotec/git2net.git'
local_directory = '.'
git_repo_dir = 'git2net4analysis'

if os.path.exists(git_repo_dir):
    shutil.rmtree(git_repo_dir)

git.Git(local_directory).clone(git_repo_url, git_repo_dir)

''

Now that we have obtained a local copy of the repository, we can use `git2net` to obtain a database containing information on all commits and edits made to obtain the current state of the repository.

To do so, we use the `mine_git_repo` function. This function takes two required inputs as well as a number of optional inputs, some of which we will further explore later in this tutorial. Let's start with the required inputs. Here, we need to supply a path to the git repositoy that will be analysed. Below, this is done with the variable `repo_name`. In addition, `git2net` requires a path to the *sqlite* database that will be filled during the mining process. This path is provided as `sqlite_db_file`.

Note, that if no database exists on the supplied path, `git2net` will create a new database. If a database exists, `git2net` will check if the database was mined with the same setting and on the same repository and subsequently resume the mining process from wherever it was left off.

Let's try this out. Below we import `git2net` and point it to the path to which we cloned the database. In addition, we specify the location of the database file in which the results of the mining process will be stored and ensure the database does currently not exist. We then run the `mine_git_repo` function with the optional argument `max_modifications = 1`. With this only commits in which 1 or less files were modified are mined.

In [2]:
import git2net

sqlite_db_file = 'git2net.db'

# Remove database if exists
if os.path.exists(sqlite_db_file):
    os.remove(sqlite_db_file)

max_modifications = 1
    
git2net.mine_git_repo(git_repo_dir, sqlite_db_file, max_modifications=max_modifications)

Found no database on provided path. Starting from scratch.


Parallel (32 processes):   0%|          | 0/169 [00:00<?, ?it/s]

Commit exceeding max_modifications:  bd0ad7b12500239321a8b7c6ba547f6111c781bb
Commit exceeding max_modifications:  eed200119f675f2abc69a5f72c3505e903b82fd2


Parallel (32 processes):   3%|▎         | 5/169 [00:00<00:50,  3.25it/s]

Commit exceeding max_modifications:  40cc53f783aeb835fbec20f4d5e165af4e24fd32
Commit exceeding max_modifications:  9a042c9c7c6a99733d1b94bf0d440f5d22389a79
Commit exceeding max_modifications:  a2d25731c924765db4f21fa3afa7d263b7c9e79d
Commit exceeding max_modifications:  c657e752b411caf531e3fff8fc0ea8e0b756ed43


...

Parallel (32 processes):  43%|████▎     | 72/169 [00:02<00:02, 40.77it/s]

Commit exceeding max_modifications:  6f9b1ad5ee73c87750ef6c1d48f3580d4b4e35ac
Commit exceeding max_modifications:  77341fec9057940ff7fd75dd780d3eec9c0b2bb1
Commit exceeding max_modifications:  c2d7f0cf365472789d4e369a4fb5ad6da5e19278
Commit exceeding max_modifications:  9d5dd27c210a0306e0bf193af6ad567e0edac0b3
Commit exceeding max_modifications:  5b6e6953445fe01ec1317a482239307fbdc2ca27
Commit exceeding max_modifications:  7df9aefe5f71d70e6bdd26b6f1c242c547be670e


Parallel (32 processes):  46%|████▌     | 78/169 [00:02<00:02, 37.63it/s]

Commit exceeding max_modifications:  2fe35a2c8001ef7f1494eb32b175baaa6615f3a5
Commit exceeding max_modifications:  7e24ad664e91ee6da32c999e66bc374023657af1
Commit exceeding max_modifications:  ed9f24ef27a185a1b4dec32777f6e7cb0491113a
Commit exceeding max_modifications:  2fe365271e5f1506a17724fbb83af5db0ad49cfa


Parallel (32 processes): 100%|██████████| 169/169 [00:07<00:00, 22.23it/s]


While mining, `git2net` provides information about the current progress. The first line shows that no database was found at the current path and mining will be started from scratch. This is totally expected, as we deliberately deleted any existing database before the run.

Subsequently, progress updates on the mining process are printed. The first information denotes the number of processes `git2net` spawns and runs on. `git2net` is highly parallelised and will automatically detect the number of threads of your CPU, fully utilising all of them during operation. In case you want to reduce this load, this can be done by specifically setting the number of processes with the `no_of_processes` option of the `mine_git_repo` function.

The other output shows the number of commits and total number of commits mined in this run, as well as the elapsed time and an estimate of the remaining time to finish.

If a commit is skipped, the reason and the commit hash are printed. Currently, there are three cases in which a commit can be skipped. Firstly, as seen above a commit can exceed the maximum number of modifications set by `max_modifications`. Secondly, processing the commit can take longer as a maximum time defined by the `timeout` option. Thirdly, a commit can be skipped due to an error occuring within the commit. In these cases, please report the repository and commit hash in a new issue on github.com/gotec/git2net.

Let's resume the mining process while increasing the maximum number of modifications to 5!

In [3]:
max_modifications = 5

git2net.mine_git_repo(git_repo_dir, sqlite_db_file, max_modifications=max_modifications)

Found a matching database on provided path. Skipping 117 (69.23%) of 169 commits. 52 commits remaining.


Parallel (32 processes):   2%|▏         | 1/52 [00:00<00:11,  4.28it/s]

Commit exceeding max_modifications:  a3213cd995e850c8966355755c4ac2ff61f65503


Parallel (32 processes):   8%|▊         | 4/52 [00:00<00:08,  5.60it/s]

Commit exceeding max_modifications:  9e72df61bf300b42c3fbc16d94153e8edbbe6dd6
Commit exceeding max_modifications:  090c00c342283134a23900f85c1d232499617365


Parallel (32 processes):  21%|██        | 11/52 [00:00<00:05,  7.72it/s]

Commit exceeding max_modifications:  6b837c8f5383d16434afa0eaaab347873113230b
Commit exceeding max_modifications:  bbc55b0ec194710e39bfe098acaf930598eb045c
Commit exceeding max_modifications:  4515874def08a49af37a4d47615a2e05dc508d89
Commit exceeding max_modifications:  3f6435c4acbf8d1b6b02e9be02b514c2c2d53446


Parallel (32 processes):  27%|██▋       | 14/52 [00:00<00:03,  9.84it/s]

Commit exceeding max_modifications:  ed7b0b70e6062c4ab9f5f66d99f3413b3b983d94


Parallel (32 processes): 100%|██████████| 52/52 [00:01<00:00, 32.74it/s]


As you can see from the output above, the process was resumed from the old database, skipping the already processed commits in the repository.

Great, we made some progress and a large amount of the commits in the repository are already mined and in the database! But what about the other ones? We get some more information on the commits missing from the database from the `mining_state_summary` function. Similar to `mine_git_repo`, it also requires the paths to the repository as well as the database.

In [4]:
git2net.mining_state_summary(git_repo_dir, sqlite_db_file)

100%|██████████| 8/8 [00:00<00:00, 40.31it/s]

161 / 169 (95.27%) of commits were successfully mined.


,hash,is_merge,modifications,author_name,author_email,author_date
0,a3213cd995e850c8966355755c4ac2ff61f65503,False,18,Christoph Gote,cgote@ethz.ch,2019-02-15 15:39:38
1,9e72df61bf300b42c3fbc16d94153e8edbbe6dd6,False,7,Christoph Gote,cgote@ethz.ch,2019-04-10 17:40:01
2,090c00c342283134a23900f85c1d232499617365,False,9,Christoph Gote,cgote@ethz.ch,2019-04-10 18:26:54
3,6b837c8f5383d16434afa0eaaab347873113230b,False,6,Christoph Gote,cgote@ethz.ch,2019-09-23 16:32:25
4,bbc55b0ec194710e39bfe098acaf930598eb045c,True,7,Christoph Gote,gotec@me.com,2019-10-03 12:02:34
5,4515874def08a49af37a4d47615a2e05dc508d89,False,15,Christoph Gote,gotec@me.com,2019-10-03 12:37:41
6,3f6435c4acbf8d1b6b02e9be02b514c2c2d53446,False,16,Christoph Gote,gotec@me.com,2019-10-03 13:02:03
7,ed7b0b70e6062c4ab9f5f66d99f3413b3b983d94,False,20,Christoph Gote,cgote@ethz.ch,2019-11-04 19:08:31


The function again provides a summary of the mining state, as well as details on all missing commits. Let's assume, we are very interest in commit *090c00c342283134a23900f85c1d232499617365* but want to avoid crawling the other missing commits. While this is uneccessary for small repositories such as `git2net` this might become higly relevant for larger projects such as `linux`, where individual commits can make changes to thousands of files which in turn require significant computational resources to analyse. This is particularly important for merge commits, as all files included in the diffs to both parent commits need to be considered. Therefore, for larger projects I generally recommend to run `git2net` with `max_modifications = 1000`, subsequently increasing this number if required.

But now back to mining specifically commit *090c00c342283134a23900f85c1d232499617365*, which can be done with the `commits` option in `mine_git_repo`. We also set the number of processes to 1, enabling serial mode, which can be very helpful for debugging as significantly more information is printed.

In [5]:
# mine_git_repo takes list of commits
commits = ['090c00c342283134a23900f85c1d232499617365']

git2net.mine_git_repo(git_repo_dir, sqlite_db_file, commits=commits, no_of_processes=1)

Serial:   0%|          | 0/1 [00:00<?, ?it/s]

Found a matching database on provided path. Skipping 161 (95.27%) of 169 commits. 8 commits remaining.


Serial: 100%|██████████| 1/1 [00:00<00:00,  2.07it/s]


Congratulations, you have now mined your first git repository using `git2net`! Note, though that not all commits have been mined at this point. This will be done at a later stage of this tutorial.

## Visualisation and Analysis

You can now use the database to query various information on different commits or edits. In addition, `git2net` also provides the functionality to generate various network projections of the data.

To start, lets try to obtain a co-editing network for our project. This is as simple as calling the `get_coediting_network` function and providing the database we just mined.

In [6]:
t, node_info, edge_info = git2net.get_coediting_network(sqlite_db_file)
t

The function returns a `pathpy` temporal network object as well as two dictionaries which can be used to return properties of nodes and edges. As of writing this tutorial not all of them are used but they are set as placeholders for future versions of `git2net`.

A `pathpy` temporal network object can be visualised by itself as shown above. In addition, we can also aggregate the network, by dropping the order of events, yielding a standard network object. Let's do this next.

In [7]:
import pathpy as pp
pp.Network.from_temporal_network(t)

In both the temporal and aggregated network, a node represents an author, whereas edges point from the person changing a line of code to the person who was the original author.

Next, we could ask the question which those files were that authors collaborated on. Therefore, we can plot a bipartite network containing both files and authors as nodes.

In [8]:
t, node_info, edge_info = git2net.get_bipartite_network(sqlite_db_file)
n = pp.Network.from_temporal_network(t)
n

For this network, `node_info` contains the classes of authors in the network. These can e.g. be used to color nodes as shown below.

In [9]:
colour_map = {'author': '#73D2DE', 'file': '#2E5EAA'}
node_color = {node: colour_map[node_info['class'][node]] for node in n.nodes}
pp.visualisation.plot(n, node_color=node_color)

The projection of this network that links authors editing the same file is the co-authorship network.

In [10]:
n, node_info, edge_info = git2net.get_coauthorship_network(sqlite_db_file)
n

Note that it looks similar, however, all information on the direction of interactions is lost.

If we are interested in e.g more recently edited files, we can filter the database by providing the `time_from` and `time_to` options. Let's check the files edited since May 2019.

In [11]:
from datetime import datetime
time_from = datetime(2019, 5, 1)
t, node_info, edge_info = git2net.get_bipartite_network(sqlite_db_file, time_from=time_from)
n = pp.Network.from_temporal_network(t)
colour_map = {'author': '#73D2DE', 'file': '#2E5EAA'}
node_color = {node: colour_map[node_info['class'][node]] for node in n.nodes}
pp.visualisation.plot(n, node_color=node_color)

`git2net` allows the extraction of editing paths on the level of individual lines. I.e. we are able to track consecutive changes made to a single line over time&mdash;even if these lines move up or down in a file, or even across files. This is very powerful, as it allows us to determine editing sequences as well as find lines that require more editing than others. These could either be very difficult lines to implement or contain very important information, such as the version number in an `__init__.py` file.

To extract these paths, we can use the `get_line_editing_paths` function. As these networks tend be very large we limit the analysis to a very small file for this tutorial. To only look at a specific set of file paths we can use the `file_paths` option.

In [12]:
file_paths = ['setup.py']
paths, dag, node_info, edge_info = git2net.get_line_editing_paths(sqlite_db_file, git_repo_dir,
                                                                  file_paths=file_paths)
pp.visualisation.plot(dag, node_color=node_info['colors'])

 53%|█████▎    | 8/15 [00:00<00:00, 72.66it/s]

Querying commits
Querying edits


100%|██████████| 47/47 [00:00<00:00, 107487.62it/s]

2020-04-22 14:05:22 [Severity.INFO]	Creating paths from directed acyclic graph
2020-04-22 14:05:22 [Severity.INFO]	Expanding Subpaths
2020-04-22 14:05:22 [Severity.INFO]	Calculating sub path statistics ... 
2020-04-22 14:05:22 [Severity.INFO]	finished.


Notice, that despite only looking at a single file the network shown above is not connected. This is due to our database not being complete. Let's fix this now and try again.

In [13]:
git2net.mine_git_repo(git_repo_dir, sqlite_db_file)

Found a matching database on provided path. Skipping 162 (95.86%) of 169 commits. 7 commits remaining.


Parallel (32 processes): 100%|██████████| 7/7 [00:07<00:00,  1.06s/it]


In [14]:
paths, dag, node_info, edge_info = git2net.get_line_editing_paths(sqlite_db_file, git_repo_dir,
                                                                  file_paths=file_paths)
pp.visualisation.plot(dag, node_color=node_info['colors'], width=1000, height=1000)

 32%|███▏      | 8/25 [00:00<00:00, 71.57it/s]

Querying commits
Querying edits


100%|██████████| 55/55 [00:00<00:00, 48210.39it/s]

2020-04-22 14:05:30 [Severity.INFO]	Creating paths from directed acyclic graph
2020-04-22 14:05:30 [Severity.INFO]	Expanding Subpaths
2020-04-22 14:05:30 [Severity.INFO]	Calculating sub path statistics ... 
2020-04-22 14:05:30 [Severity.INFO]	finished.


As mentioned before, these networks get very large very quickly. Therefore, it is often more useful to work with the `pathpy` path object that is also returned by the function. It cointains all paths and subpaths contained in the network shown above. More information regarding this object can be found in the documentation on [pathpy.net](http://www.pathpy.net/).

This concludes this tutorial, which I hope you found useful. Enjoy using `git2net` and best of luck for your research! If you find any bugs with the code please let me know on [github.com](https://github.com/gotec/git2net).

`git2net` has been developed as open source project. This means your ideas and inputs are highly welcome. Feel free to share the project and contribute yourself. You can imediately get started on the repository you just downloaded!